In [ ]:
import json

import numpy as np
import pandas as pd
import pymongo
import scipy
from bson import ObjectId, json_util
from pandas.io.json import json_normalize
from pymongo import MongoClient as Connection
from scipy import sparse, spatial
from scipy.spatial.distance import cdist, pdist, squareform
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:

connection = Connection()
db = connection.recipe_db
input_data = db.recipes_test_copy
#data = pd.DataFrame(list(input_data.find()))

data = json.loads(json_util.dumps(input_data.find()))
norm_data = json_normalize(data)

ingredients = json_normalize(data, record_path='ingredients', meta='id', record_prefix='ingredients_', errors='ignore' )
ingredients = pd.DataFrame(ingredients)

#schmeiss alle zutaten raus, die weniger als 4 mal verwendet werden
# ingredients_unique = ingredients[ingredients.groupby('ingredients_id').ingredients_id.transform(len) > 4]

# drop duplicate by a column name
 
# ingredients_unique.drop_duplicates(['ingredients_id'], keep='last')

# ingredients_unique = ingredients_unique.drop(ingredients_unique.columns[[0, 3, 4]], axis=1)

# ingredients_unique['ingredients_id'].value_counts()



In [ ]:
norm_data

In [ ]:
nutritions = json_normalize(data, record_path='nutritions',
                            meta=['id', 'prep_time', 'rating', 'rating_count', 'ready_in_time', 'review_count'])


In [183]:
nutritions.id.nunique()

6305

In [ ]:
v = nutritions.pivot_table(
        index=['id'], 
         columns='name', 
         values='amount'
).reset_index()

v.index.name = v.columns.name = None

v.set_index('id', inplace=True)
v = v.apply(pd.to_numeric)
v = v.fillna(0)
v

In [ ]:
# data cleansing

# schmeiss alle zutaten raus, die weniger als 5 mal verwendet werden
ingredients_filt = ingredients.groupby('ingredients_id').filter(lambda x: len(x) >= 5)

# drop alls rows with ingredients_id == 0
ingredients_filt = ingredients_filt[ingredients_filt.ingredients_id != 0]

# drop all duplicate
ingredients_filt = ingredients_filt.drop_duplicates(keep='first')


In [184]:
# data cleansing

# schmeiss alle zutaten raus, die weniger als 5 mal verwendet werden
ingredients_filt = ingredients.groupby('ingredients_id').filter(lambda x: len(x) >= 5)

# drop alls rows with ingredients_id == 0
ingredients_filt = ingredients_filt[ingredients_filt.ingredients_id != 0]

# drop all duplicate
ingredients_filt = ingredients_filt.drop_duplicates(keep='first')
ingredients_filt.id.nunique()

6305

In [ ]:
id_list = [215014, 8669, 16700, 16354, 12720, 8652, 8887, 51283, 45954, 213742, 14595, 219164, 16348, 143082, 8665, 11758, 223042, 236609, 8778, 65896, 24264, 11679, 141678, 9023]


recipe_db = pd.get_dummies(ingredients_filt['ingredients_id']).groupby(ingredients_filt['id']).apply(max)



In [ ]:
recipe_db.index.nunique()

In [186]:
#filtere Rezepte aus den Metadaten anhand ihrerr Zutatenmenge raus. 
not_wanted_recipes = recipe_db[recipe_db.mask(recipe_db==0).count(axis=1) <= 4]
mask = nutritions['id'].isin(not_wanted_recipes.index.values)
not_wanted_recipes

,111,126,257,388,443,445,578,615,629,631,...,24850,24865,25518,25522,26269,26706,26934,27343,27388,27805
id,,,,,,,,,,,,,,,,,,,,,


In [187]:

nutritions_filt = nutritions.loc[~mask]
# nutritions_filt.id.nunique()
v = nutritions_filt.pivot_table(
        index=['id'], 
         columns=['name'],
         values=['amount'],
).reset_index()

v.set_index('id', inplace=True)
nutritions_filt.set_index('id', inplace=True)
nutritions_filt

,amount,display_value,name,percent_daily_value,unit,prep_time,rating,rating_count,ready_in_time,review_count
id,,,,,,,,,,
59661,35.958460,36,Fat,55,g,20,4.335593,1180,40,817
59661,509.802900,510,Calories,25,kcal,20,4.335593,1180,40,817
59661,95.274040,95,Cholesterol,32,mg,20,4.335593,1180,40,817
59661,353.601100,354,Sodium,14,mg,20,4.335593,1180,40,817
59661,32.266050,32.3,Carbohydrates,10,g,20,4.335593,1180,40,817
59661,18.239260,18.2,Protein,36,g,20,4.335593,1180,40,817
59661,109.678900,110,Folate,61,mcg,20,4.335593,1180,40,817
59661,105.731700,106,Magnesium,38,mg,20,4.335593,1180,40,817
59661,0.331828,< 1,Vitamin B6,21,mg,20,4.335593,1180,40,817


In [ ]:
ready_in_time = nutritions_filt.pivot_table(
        index=['id'], 
#          columns=['ready_in_time'],
         values=['ready_in_time'],
).reset_index()

# v.index.name = v.columns.name = None

ready_in_time.set_index('id', inplace=True)
# ready_in_time

prep_time = nutritions_filt.pivot_table(
        index=['id'], 
#          columns=['ready_in_time'],
         values=['prep_time'],
).reset_index()

# v.index.name = v.columns.name = None

prep_time.set_index('id', inplace=True)
# prep_time

prep_time[prep_time == 0].count()
ready_in_time[ready_in_time == 0].count()

In [ ]:

# v.reset_index()
# nutritions_filt.reset_index()
v_new = pd.concat([v, nutritions_filt['prep_time']], axis=1)

# v_new = v_new.set_index('id', inplace=True)
v_new

In [188]:
#filter alle rezepte, die mehr gleich/als 5 rezepte haben
recipe_db = recipe_db[recipe_db.mask(recipe_db==0).count(axis=1) >= 5]

recipe_db.index.nunique()
user_recipes = recipe_db[recipe_db.index.isin(id_list)]
user_recipes

5789

,111,126,257,388,443,445,578,615,629,631,...,24850,24865,25518,25522,26269,26706,26934,27343,27388,27805
id,,,,,,,,,,,,,,,,,,,,,
8652,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8665,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8669,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8778,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8887,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9023,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11679,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11758,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12720,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
### Delta Methode - Rezept zu Rezeptvergleich

# result_1 = pd.DataFrame()

# for index, recipe in recipe_db.iterrows():
#     user_recipes_subtracted = user_recipes.sub(recipe)
#     recipe = pd.Series({'delta':user_recipes_subtracted.abs().values.sum()}, name=index)
#     result_1 = result_1.append(recipe)

# result_1 = result_1.sort_values(by='delta')
#result_1


In [ ]:
##Euklidische Distanz - rezept zu rezept vergelich

result_array = cdist(user_recipes, recipe_db,'euclidean')
result_2 = pd.DataFrame(result_array, columns=recipe_db.index.values, index=user_recipes.index.values)
# result_2

result_3 = pd.DataFrame(result_2.sum(), columns=['euclidiean_sum'])
result_3 = result_3.sort_values(by='euclidiean_sum')

result_3

In [ ]:
##Minkowski Distanz - rezept zu rezept vergelich

result_array = cdist(user_recipes, recipe_db,'minkowski', p = 2)
result_2 = pd.DataFrame(result_array, columns=recipe_db.index.values, index=user_recipes.index.values)
# result_2

result_8 = pd.DataFrame(result_2.sum(), columns=['euclidiean_sum'])
result_8 = result_8.sort_values(by='euclidiean_sum')

result_8

In [ ]:
##Euklidische Distanz - rezept zu rezept vergelich

result_array = cdist(user_recipes, recipe_db,'jaccard')
result_2 = pd.DataFrame(result_array, columns=recipe_db.index.values, index=user_recipes.index.values)
# result_2

result_6 = pd.DataFrame(result_2.sum(), columns=['jaccard_distance_sum'])
result_6 = result_6.sort_values(by='jaccard_distance_sum')

#result_6

In [ ]:
##Euklidische Distanz - rezept zu rezept vergelich

result_array = cdist(user_recipes, recipe_db,'cityblock')
result_2 = pd.DataFrame(result_array, columns=recipe_db.index.values, index=user_recipes.index.values)
# result_2

result_7 = pd.DataFrame(result_2.sum(), columns=['manhattan_distance (delta-Methode)'])
result_7 = result_7.sort_values(by='manhattan_distance (delta-Methode)')

#result_7

In [ ]:
##Euklidische Distanz - Userpräferenz
#1. Array mit user Rezepten zusammenstellen (am besten mit index, sodass jedes Rezept eine feste nummer hat)
#2. Aus allen Rezepten des Arrays einen Userpräferenz Vektor bilden (Rezeptpräsenz wird aufadiert)

# user_pref = pd.DataFrame(index=['user_pref'])
# user_pref.loc['user_pref'] = user_recipes.sum()
# distance_array = cdist(user_pref.loc[['user_pref']], recipe_db, metric='euclidean')
# result_4 = pd.DataFrame(distance_array, columns=recipe_db.index.values, index=['euclidiean_user_pref'])
# result_4 = result_4.T.sort_values(by='euclidiean_user_pref')
# result_4

In [ ]:

result_array = cosine_similarity(user_recipes, recipe_db)
result_2 = pd.DataFrame(result_array, columns=recipe_db.index.values, index=user_recipes.index.values)
result_5 = pd.DataFrame(result_2.sum(), columns=['cosine_distance_sum'])
result_5 = result_5.sort_values(by='cosine_distance_sum')
result_5 = result_5.sort_values(ascending=False, by='cosine_distance_sum')

In [ ]:
display_side_by_side(result_7, result_3, result_5, result_6)


In [ ]:

# result_array = cosine_similarity(user_recipes, recipe_db)
# result_2 = pd.DataFrame(result_array, columns=recipe_db.index.values, index=user_recipes.index.values)
# result_3 = pd.DataFrame(result_2.sum(), columns=['distance_sum'])
# result_3 = result_3.sort_values(by='distance_sum')
#result_3.sort_values(ascending=False, by='distance_sum')

In [ ]:
top10 = result_6[:10].index.values

#überschneidung jaccard und euklidische distanz top 100 
jac = pd.Series(result_6[:100].index.values)
euc = pd.Series(result_3[:100].index.values)

pd.Series(list(set(jac).intersection(set(euc))))